In [2]:
import statsmodels.api as sm
import pandas as pd #librería para manejo de datos
import statsmodels.stats.api as sms
from statsmodels.compat import lzip

datos_f1=pd.read_csv('datos/Datos F1_nuevo.csv', sep=';', decimal=',') 
X=datos_f1.values[:,1:13]
X = X.astype('float64')
y=datos_f1.values[:,0]
y = y.astype('float64')
#print(X)
mco1 = sm.OLS(y, sm.add_constant(X)).fit()

mco1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     28.01
Date:                Tue, 23 Nov 2021   Prob (F-statistic):           4.69e-18
Time:                        08:54:30   Log-Likelihood:                -184.83
No. Observations:                  63   AIC:                             395.7
Df Residuals:                      50   BIC:                             423.5
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         19.5946     14.814      1.323      0.192     -10.160      49.349
x1            11.4177      1.916      5.959      0.000       7.569      15.266
x2             2.9419      0.516      5.699      0.000       1.905       3.979
x3            -2.1852      0.793     -2.755      0.008      -3.778      -0.592
x4             0.7530      0.501      1.502      0.139      -0.254       1.760
x5            -0.0915      0.061     -1.509      0.138      -0.213       0.030
x6            -0.1015      0.333     -0.305      0.762      -0.770       0.567
x7             0.0149      0.009      1.604      0.115      -0.004       0.034
x8            -0.3179      1.363     -0.233      0.817      -3.056       2.421
x9            -1.4723      1.206     -1.221      0.228      -3.894       0.949
x10            1.3061      1.320      0.989      0.327      -1.346       3.958
x11            4.1089      1.564      2.627      0.011       0.967       7.251
x12           -3.0421      0.952     -3.194      0.002      -4.955      -1.129
==============================================================================
Omnibus:                       36.889   Durbin-Watson:                   2.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               92.817
Skew:                           1.843   Prob(JB):                     7.00e-21
Kurtosis:                       7.667   Cond. No.                     8.32e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.32e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""